In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

# Load datasets
flights = pd.read_csv('../data/ontime.csv')
flights

,year,quarter,month,dayofmonth,dayofweek,flightdate,reporting_airline,dot_id_reporting_airline,iata_code_reporting_airline,tail_number,flight_number_reporting_airline,originairportid,originairportseqid,origincitymarketid,origin,origincityname,originstate,originstatefips,originstatename,originwac,destairportid,destairportseqid,destcitymarketid,dest,destcityname,deststate,deststatefips,deststatename,destwac,crsdeptime,deptime,depdelay,depdelayminutes,depdel15,departuredelaygroups,deptimeblk,taxiout,wheelsoff,wheelson,taxiin,crsarrtime,arrtime,arrdelay,arrdelayminutes,arrdel15,arrivaldelaygroups,arrtimeblk,cancelled,cancellationcode,diverted,crselapsedtime,actualelapsedtime,airtime,flights,distance,distancegroup,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay,firstdeptime,totaladdgtime,longestaddgtime,divairportlandings,divreacheddest,divactualelapsedtime,divarrdelay,divdistance,divairport
0,2013,1,1,27,7,2013-01-27,9E,20363,9E,N907XJ,3341,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13487,1348702,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,1821,1823,2,2,0,0,1800-1859,21,1844,2041,16,2049,2057,8,8,0,0,2000-2059,0,0,0,148,154,117,1,852,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2013,1,1,28,1,2013-01-28,9E,20363,9E,N923XJ,3341,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13487,1348702,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,1821,1900,39,39,1,2,1800-1859,16,1916,2104,6,2049,2110,21,21,1,1,2000-2059,0,0,0,148,130,108,1,852,4,0,0,0,0,21,0,0,0,0,0,0,0,0,0
2,2013,1,1,29,2,2013-01-29,9E,20363,9E,N912XJ,3341,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13487,1348702,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,1821,1947,86,86,1,5,1800-1859,17,2004,2136,5,2049,2141,52,52,1,3,2000-2059,0,0,0,148,114,92,1,852,4,0,0,0,0,52,0,0,0,0,0,0,0,0,0
3,2013,1,1,30,3,2013-01-30,9E,20363,9E,N931XJ,3341,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13487,1348702,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,1821,1859,38,38,1,2,1800-1859,12,1911,2106,8,2049,2114,25,25,1,1,2000-2059,0,0,0,148,135,115,1,852,4,0,0,0,0,25,0,0,0,0,0,0,0,0,0
4,2013,1,1,31,4,2013-01-31,9E,20363,9E,N919XJ,3341,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13487,1348702,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,1821,1906,45,45,1,3,1800-1859,19,1925,2118,5,2049,2123,34,34,1,2,2000-2059,0,0,0,148,137,113,1,852,4,4,0,0,0,30,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498082,2013,1,1,8,2,2013-01-08,WN,19393,WN,N725SW,3758,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1215,1220,5,5,0,0,1200-1259,26,1246,1336,7,1340,1343,3,3,0,0,1300-1359,0,0,0,145,143,110,1,862,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
498083,2013,1,1,8,2,2013-01-08,WN,19393,WN,N409WN,389,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,1155,1211,16,16,1,1,1100-1159,8,1219,1733,7,1735,1740,5,5,0,0,1700-1759,0,0,0,220,209,194,1,1620,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
498084,2013,1,1,8,2,2013-01-08,WN,19393,WN,N294WN,2782,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,1655,1717,22,22,1,1,1600-1659,8,1725,2223,5,2235,2228,-7,0,0,-1,2200-2259,0,0,0,220,191,178,1,1620,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
498085,2013,1,1,8,2,2013-01-08,WN,19393,WN,N600WN,268,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,13198,1319801,33198,MCI,"Kansas City, MO",MO,29,Missouri,64,1850,1853,3,3,0,0,1800-1859,16,1909,2119,5,2120,2124,4,4,0,0,2100-2159,0,0,0,90,91,70,1,533,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
aircraft = pd.read_csv('../data/aircraft.csv')
aircraft

,tail_number,manufacturer,model
0,N789SW,Boeing,B-737-7H4
1,N198UW,Airbus,A321-211
2,N681UP,Boeing,B747-1
3,N54711,Boeing,B737-700PAX
4,N390SW,Boeing,B-737-3H4
...,...,...,...
13096,N923TW,McDonnell-Douglas,SUPER80PASSENGER
13097,N379WN,Boeing,737-300PASSENGERONLY
13098,N821NN,Boeing,B737-823PASSENGER
13099,N247LR,Bombardier,CRJ-900-PSGR


In [4]:
airport = pd.read_csv('../data/airport_id.csv')
airport

,code,description
0,11712,"Greven, Germany: Munster/Osnabruck"
1,11713,"Fort Madison, IA: Fort Madison Municipal"
2,11714,"Fort Myers, FL: Page Field"
3,11715,"Freetown, Sierra Leone: Freetown Lungi"
4,11716,"Funchal, Portugal: Madeira"
...,...,...
6245,12878,"Lafayette, IN: Purdue University"
6246,12879,"Lannion, France: Lannion Airport"
6247,12880,"Lages, Brazil: Lages Airport"
6248,12881,"Aklavik, Canada: Aklavik Airport"


In [5]:
# Ensure 'flightdate' is in datetime format for extracting day of the week and time of day
flights['flightdate'] = pd.to_datetime(flights['flightdate'])
flights['day_of_week'] = flights['flightdate'].dt.dayofweek  # Monday=0, Sunday=6
flights['time_of_day'] = flights['crsdeptime'] // 100  # Simplified hour of the day

In [6]:
# Merge aircraft manufacturer info
flights = flights.merge(aircraft[['tail_number', 'manufacturer']], on='tail_number', how='left')

# Calculate average delay by Chicago-related airport, airline, and manufacturer
airport_delays = flights.groupby('origin')['depdelay'].mean().reset_index().rename(columns={'depdelay': 'avg_airport_delay'})
airline_delays = flights.groupby('reporting_airline')['depdelay'].mean().reset_index().rename(columns={'depdelay': 'avg_airline_delay'})
manufacturer_delays = flights.groupby('manufacturer')['depdelay'].mean().reset_index().rename(columns={'depdelay': 'avg_manufacturer_delay'})

# Merge these averages back into the main DataFrame
flights = flights.merge(airport_delays, how='left', on='origin')
flights = flights.merge(airline_delays, how='left', on='reporting_airline')
flights = flights.merge(manufacturer_delays, on='manufacturer', how='left')

In [7]:
# Define delay categories based on the simplified distribution of delays
# Flights that are diverted or significantly delayed (>60 minutes) are automatically considered a 'Long Delay'
flights['delay_category'] = pd.cut(flights['arrdelay'], bins=[flights['arrdelay'].min(), 0, 15, flights['arrdelay'].max()], 
                                   labels=[0, 1, 2], include_lowest=True)

# Assign the highest category for diverted flights or any other condition you consider as severe
flights.loc[(flights['diverted'] == 1) | (flights['arrdelay'] > 15), 'delay_category'] = 2

In [8]:
# chicago_airport_codes = airport[airport['description'].str.contains("Chicago")]['code']

# # Filter the original dataset for flights that either arrive at or depart from Chicago
# chicago_flights = flights[flights['destairportid'].isin(chicago_airport_codes)]
# chicago_flights
chicago_flights = flights.copy()

In [9]:
# Select necessary features and the target
X = chicago_flights[['avg_airport_delay', 'avg_airline_delay', 'avg_manufacturer_delay', 'time_of_day', 'day_of_week']]
X

,avg_airport_delay,avg_airline_delay,avg_manufacturer_delay,time_of_day,day_of_week
0,10.123022,8.136415,7.322180,18,6
1,10.123022,8.136415,7.322180,18,0
2,10.123022,8.136415,7.322180,18,1
3,10.123022,8.136415,7.322180,18,2
4,10.123022,8.136415,7.322180,18,3
...,...,...,...,...,...
498082,11.169087,7.187755,5.690283,12,1
498083,11.169087,7.187755,5.690283,11,1
498084,11.169087,7.187755,5.690283,16,1
498085,11.169087,7.187755,5.690283,18,1


In [10]:
y = chicago_flights['delay_category']
y

0         1
1         2
2         2
3         2
4         2
         ..
498082    1
498083    1
498084    0
498085    1
498086    1
Name: delay_category, Length: 498087, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [11]:
# Adjust class weights inversely proportional to class frequencies
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Fill missing values with the median of each column
X.fillna(X.median(), inplace=True)

# Creating polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Split the enhanced dataset
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, stratify=y, random_state=42)

# Compute class weights for the classes actually present in y_train
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {c: w for c, w in zip(np.unique(y_train), class_weights)}

# Train the model with class weight adjustment
classifier = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight=class_weight_dict, random_state=42)
classifier.fit(X_train, y_train)

# Predict on the test set and evaluate
y_pred = classifier.predict(X_test)

# Evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.49      0.59     63792
           1       0.24      0.28      0.26     19091
           2       0.25      0.54      0.34     16735

    accuracy                           0.46     99618
   macro avg       0.41      0.43      0.40     99618
weighted avg       0.56      0.46      0.48     99618

Confusion Matrix:
[[31188 13193 19411]
 [ 6880  5354  6857]
 [ 4328  3450  8957]]
Accuracy: 0.45673472665582526
